# CSC 421 - Constraint Satisfaction Problems

### Instructor: Shengyao Lu

We have used **atomic** representation to solve <mark>state-space search problems</mark> by searching in trees or graphs. We also found in informed search algorithms that <mark>domain-specific heuristics could estimate the cost</mark> of reaching the goal from a given state. 

In today's class, we will look into problems using a **factored representation** for each state, which refers to a set of variables, each of which has a value. In these problems, the structure of the states is somewhat awared, and use general rather than domain-specific heuristics. 

## 1. Defining Constraint Satisfaction Problems (CSP)

A CSP problem consists of three components, $\mathcal{X,D,C}$.
- $\mathcal{X}$ := a set of variables, $\{X_1, \dots, X_n\}$.
- $\mathcal{D}$ := a set of domains, $\{D_1, \dots, D_n\}$.
    - A domain $D_i$ consists of a set of allowable values $\{v_1\dots v_k\}$ for variable $X_i$. $k$ := dimension of $D_i$, different variables can have different domains of different sizes. 
    - e.g., a boolean variable $X_{bool}$ have the domain $\{true, false\}$
- $\mathcal{C}$ := a set of constraints that specify allowable combinations of values.
    - Each constraint $C_j$ consists of a pair $\left\langle scope,rel \right\rangle$.
        - $scope$ := a tuple of variables in $C_j$.
        - $rel$ := a relation that defines the values that $scope$ can take on. 
    - e.g., if $X_1,X_2$ both have the domain $\{1,2,3\}$, then two ways to represent the constraint $C_m$ are, where $C_m$ := $X_1$ must be greater than $X_2$:
        - $\left\langle (X_1,X_2), \{(3,1), (3,2), (2,1)\}\right\rangle$
        - $\left\langle (X_1,X_2),X_1>X_2\right\rangle$

### Terminology 

- **Assignment of values to variables:** e.g. $\{X_i=v_i, X_j=v_j,\dots\}$. 
- **Complete assignment:** each variable is assigned a value.  
- **Consistent (legal) assignment:** Not violate any constraints.
- **Solution to a CSP:** a consistent, complete assignment. 
- **Partial assignment:** leaves some variables unassigned.
- **Partial solution:** a consistent partial assignment.


### Example problem: Midterm schedule

Now consider we are going to have our midterm for this course on Thursday, Feb 26, 2026, at the Computer-Based Testing Lab (CBTL). There will be 4 time slots for the students to choose. You may choose whichever time slot that works for you best. However, we are not the only course where the midterm will take place on Feb 26. Some other courses may include OS (Operating System), DB (database), ALG (Algorithms), MATH, and we are the AI course. 

Assume